In [1]:
import scanpy as sc
import pandas as pd
from sklearn.svm import LinearSVC
import convex_gating as cg
from convex_gating import tools as ct
from convex_gating import helper as ch
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score



In [2]:
locations = ['Asymptotic/asymptotic.h5ad',
            'Mild/mild.h5ad',
            'Moderate/moderate.h5ad',
            'Severe/severe.h5ad',
            'Healthy/healthy.h5ad',
            'Critical/critical.h5ad',
            'Non_cov/non_cov.h5ad']

cluster_string = 'initial_clustering'
targets = list(pd.read_csv('target_clusters_sccoda.csv')['credible_effect_clusters'])
cols = ['sample','celltype','f1','recall','precision']
linear_svm_df = pd.DataFrame(columns=cols)
for loc in locations:
    sample = sc.read_h5ad(loc)
    acronym = loc.split('/')[0]
    cell_data = ch.preprocess_adata_gating(sample,cluster_string)
    cell_data = cell_data.drop("cell_ID",axis=1)
    for target_cluster in targets:
        y_true = (cell_data['initial_clustering'] == target_cluster).values*1
        if sum(y_true) != 0:
            X_true = cell_data.drop("initial_clustering",axis=1).values
            linear_svm = LinearSVC()
            linear_svm.fit(X_true,y_true)
            y_pred = linear_svm.predict(X_true)
            f1 = f1_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred)
            precision = precision_score(y_true, y_pred)
            ser = pd.Series((acronym,target_cluster,f1,recall,precision),index = cols)
            linear_svm_df = linear_svm_df.append(ser,ignore_index=True)
        else:
            ser = pd.Series((acronym,target_cluster,'na','na','na'),index = cols)
            linear_svm_df = linear_svm_df.append(ser,ignore_index=True)
linear_svm_df.to_csv('linear_svm_overview.csv')
        
        
    
    